In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math
from math import log

import pprint

In [2]:
# 书上题目5.1
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

In [3]:
datasets, labels = create_data()

In [4]:
train_data = pd.DataFrame(datasets, columns=labels)

In [5]:
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [6]:
# 熵
def ent(datasets):
    data_len = len(datasets)
    label_cnt = dict()
    for i in range(data_len):
        label = datasets[i][-1]
        if label not in label_cnt:
            label_cnt[label] = 0
        label_cnt[label] += 1
    ent = -sum([(p / data_len) * log(p / data_len, 2) for p in label_cnt.values()])
    return ent

# 经验条件熵
def cond_ent(datasets, axis=0):
    data_len = len(datasets)
    feature_sets = dict()
    for i in range(data_len):
        feature = datasets[i][axis]
        if feature not in feature_sets:
            feature_sets[feature] = list()
        feature_sets[feature].append(datasets[i])
    cond_ent = sum([(len(p) / data_len) * ent(p) for p in feature_sets.values()])
    return cond_ent

# 信息增益
def info_gain(ent, cond_ent):
    return ent - cond_ent

def info_gain_train(datasets):
    ent_ = ent(datasets)
    best_feature = list()
    for c in range(len(datasets[0]) - 1):
        c_info_gain = info_gain(ent_, cond_ent(datasets, axis=c))
        best_feature.append((c, c_info_gain))
        print('特征({}) - 信息增益：{:.4f}'.format(labels[c], c_info_gain))
    bf = max(best_feature, key=lambda x: x[-1])
    return '特征({})的信息增益最大，选择为根节点特征'.format(labels[bf[0]])

In [7]:
info_gain_train(np.array(datasets))

特征(年龄) - 信息增益：0.0830
特征(有工作) - 信息增益：0.3237
特征(有自己的房子) - 信息增益：0.4200
特征(信贷情况) - 信息增益：0.3630


'特征(有自己的房子)的信息增益最大，选择为根节点特征'

In [8]:
# 定义节点类 二叉树
class Node(object):
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = dict()
        self.result = {'label': self.label, 'feature': self.feature, 'tree': self.tree}
        
    def __repr__(self):
        return '{}'.format(self.result)
    
    def add_node(self, val, node):
        self.tree[val] = node
        
    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)
    
class DTree(object):
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = dict()
       
    # 熵
    @staticmethod
    def ent(datasets):
        data_len = len(datasets)
        label_cnt = dict()
        for i in range(data_len):
            label = datasets[i][-1]
            if label not in label_cnt:
                label_cnt[label] = 0
            label_cnt[label] += 1
        ent = -sum([(p / data_len) * log(p / data_len, 2) for p in label_cnt.values()])
        return ent
    
    # 经验条件熵
    def cond_ent(self, datasets, axis=0):
        data_len = len(datasets)
        feature_sets = dict()
        for i in range(data_len):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = list()
            feature_sets[feature].append(datasets[i])
        cond_ent = sum([(len(p) / data_len) * self.ent(p) for p in feature_sets.values()])
        return cond_ent
    
    # 信息增益
    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent
    
    def info_gain_train(self, datasets):
        ent_ = ent(datasets)
        best_feature = list()
        for c in range(len(datasets[0]) - 1):
            c_info_gain = self.info_gain(ent_, self.cond_ent(datasets, axis=c))
            best_feature.append((c, c_info_gain))
        return max(best_feature, key=lambda x: x[-1])
    
    def train(self, train_data):
        """
        input:数据集D(DataFrame格式)，特征集A，阈值eta
        output:决策树T
        """
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns
        
        # (1) 若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T
        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])
        
        # (2) 若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
        if len(features) == 0:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        # (3) 计算最大信息增益 同5.1,Ag为信息增益最大的特征
        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]
        
        # (4) Ag的信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        # (5) 构建Ag子集
        node_tree = Node(root=False, feature_name=max_feature_name, feature=max_feature)
        
        feature_list = train_data[max_feature_name].value_counts().index
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)
            # (6) 递归生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)
        return node_tree
    
    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree
    
    def predict(self, X_test):
        return self._tree.predict(X_test)

In [9]:
datasets, labels = create_data()
df = pd.DataFrame(datasets, columns=labels)
dt = DTree()
tree = dt.fit(df)

In [10]:
tree

{'label': None, 'feature': 2, 'tree': {'否': {'label': None, 'feature': 1, 'tree': {'否': {'label': '否', 'feature': None, 'tree': {}}, '是': {'label': '是', 'feature': None, 'tree': {}}}}, '是': {'label': '是', 'feature': None, 'tree': {}}}}

In [11]:
dt.predict(['老年', '否', '否', '一般'])

'否'

In [12]:
# sklearn
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    # print(data)
    return data[:,:2], data[:,-1]

X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train,)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [15]:
clf.score(X_test, y_test)

1.0